In [1]:
import pandas as pd
import numpy as np
import statistics
import matplotlib as plt

**Get input for file**

- below are two inputs
    - data 
    - samples
- these inputs take in the excel file and read the data assigning the amount of cells [samples] needed to read in later lines

- the excel is turned into a data frame and the data is displayed below

In [2]:
data = input("Please Enter File name with extension: ") #TODO: change this and line 32 to be taken in from the gui 
samples = int(input("Please Enter number of samples: "))
exceldata = pd.read_excel(data)
df = pd.DataFrame(exceldata)
df

,Unnamed: 0,Temperature(¡C),1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,STD,454.071,284.94,147.609,81.266,39.527,19.359,11.515,5.574,3.377,2.075,1.772,0.973
1,NaN,HOT,74.511,25.078,2.019,1.476,2.278,2.031,1.621,2.012,1.888,1.913,1.979,2.002
2,NaN,COLD,13.498,8.007,2.005,1.886,1.886,1.788,1.841,2.021,2.277,2.153,1.903,2.197
3,NaN,NaN,CAG-OCA1,VMD2-OCA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Creation of standard variables**

*important globals for calculations*
- dfHOT = the hot standard samples needed for titer calculations
- dfCOLD = the cold standard samples need for titer calculations

*Type 1*
- DNAdf = The indented serial dilution of the DNA standard 
- dfSTD = The actual experimental serial dilution of the AAV DNA standard

*type 2*
- DNAdf2 = the intended serial dilution but not including 100 if the r^2 value is better for this result 
- dfSTD2 = the acctual experimental serial dilution of the AAV DNA standard again just not including 100 

*creating numpy arrays*
- econc = DNAdf array
- aconc = dfSTD array
- econc2 = DNAdf2 array
- aconc2 = dfSTD2 array
- hot = dfHOT array
- cold = dfCOLD array

In [3]:
##################################################################################################
DNAdf = [100, 50, 25, 12.5, 6.25, 3.125, 1.56, 0.78, 0.39, 0.195, 0.1, 0] #serial dilution df
dfSTD = df.iloc[0, 2:] #extracts STD data into new DF
dfHOT = df.iloc[1, 2:2+samples] #extracts hot data into new DF
dfCOLD = df.iloc[2, 2:2+samples] #extracts cold Data into new DF 
##################################################################################################
DNAdf2 = [50, 25, 12.5, 6.25, 3.125, 1.56, 0.78, 0.39, 0.195, 0.1, 0] #serial dilution for calculating based on only 11 values (calculate better r value)
dfSTD2 = df.iloc[0, 3:] #extract viral df to match with the serial dilution length
econc2 = np.array(DNAdf2, dtype=np.float64) #expected DNA concentration per sample 
aconc2 = np.array(dfSTD2, dtype=np.float64) #actual DNA concentration per sample 
##################################################################################################
econc = np.array(DNAdf, dtype=np.float64) #expected DNA concentration per sample 
aconc = np.array(dfSTD, dtype=np.float64) #actual values
hot = np.array(dfHOT, dtype=np.float64) #hot samples 
cold = np.array(dfCOLD, dtype=np.float64) #cold samples 
##################################################################################################

print(hot)
print(cold)

[74.511 25.078]
[13.498  8.007]


**creating functions for calculations**

In [4]:
def rSquared(act_results, expected_results, slope, intercept): #function that determines the r square value
    ss_error = 0 #sum squared error 
    ss_tot = 0 #sum squared total
    for i in range(len(aconc-1)): 
        ss_error += (act_results[i] - slope * expected_results[i] + intercept)**2 #from index 0 to 11 take that array value and subract the line of best fit position 
        ss_tot += ( aconc[i]-statistics.mean(aconc))**2 # for each point in the array subtract the mean from it
    r_square = round(1 - (ss_error/ss_tot), 4) # r squared = (1 - (ss_error/ss_tot)) then round to 4 decimal points
    return r_square # return the r square value 


def compareR(r1, r2): # a function that compares the r square values to determine the better
    if(r1>r2):
        return r1
    elif(r1 == r2):
        return r1
    else:
        return r2


def titer(sample_difference, slope, intercept): #function that calculates the actual titer of the virus 
    titerlist = []
    for i in range(len(sample_difference)):
        reshape = (sample_difference[i] - intercept )/slope
        titer = scientific_notation="{:e}".format((15*reshape*360000000*(15/13.5))/0.01)
        titerlist.append(titer)
    return titerlist
    


**Final Calculations**

In [5]:
def calculate():
    diff =[]
    if len(econc) == len(aconc):
        for conc in range(len(aconc)):
            aconc[conc] = aconc[conc] - aconc[len(aconc)-1] #reshapes the data by subtracting irrelevant noise from the samples (ie cell 12 where there should be no DNA conc.)

    if len(hot) == len(cold):
        for sample in range(len(hot)):
            diff.append(hot[sample] - cold[sample]) # take the difference between hot and cold and append that difference to a list
    
    hcDiff = np.array(diff, dtype=np.float64) #turn the list into an np array
    slope, intercept = np.polyfit(econc, aconc, 1)

    return (titer(hcDiff, slope, intercept)) ##TODO: change this 

calculate()

['6.726481e+12', '1.134795e+12']